In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Obtención de los datos

In [19]:
# Leer los datos
data = pd.read_csv('datos_2023.csv', delimiter=';', encoding='ISO-8859-1', dtype={'superficie_km2': float})

# Filtra los datos para obtener solo los municipios
municipios_data = data[data['Serie'] == 'Municipios'][data['Nombre'] != 'Madrid']

# Selecciona las columnas deseadas
selected_columns = ['superficie_km2', 
                    'contratos_primario_temp_2023', 'contratos_primario_indefinidos_2023', 
                    'contratos_primario_total_2023', 'contratos_secundario_temp_2023', 
                    'contratos_secundario_indefinidos_2023', 'contratos_secundario_total_2023',
                    'contratos_terciario_temp_2023', 'contratos_terciario_indefinidos_2023', 
                    'contratos_terciario_total_2023', 'contratos_porcentaje_indefinidos_2023',
                    'contratos_porcentaje_temp_2023', 'contratos_total_registrados_2023', 'ocupados_primario_2023',
                    'ocupados_secundario_2023', 'ocupados_terciario_2023', 'sup_explotaciones_agricolas_total', 
                    'sup_explotaciones_agricolas_huertos_consumo_propio', 'sup_total_indust', 'unidades_productivas_primario_2023',
                    'unidades_productivas_secundario_2023', 'unidades_productivas_terciario_2023']

# Selecciona las columnas deseadas del DataFrame de municipios
clustering_data = municipios_data[selected_columns]

C:\Users\ana\AppData\Local\Temp\ipykernel_2604\66908939.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  municipios_data = data[data['Serie'] == 'Municipios'][data['Nombre'] != 'Madrid']


## EDA

In [20]:
# Crear las nuevas características
sup_utiles_ha = clustering_data['sup_explotaciones_agricolas_total'] - clustering_data['sup_explotaciones_agricolas_huertos_consumo_propio']
sup_utiles_agric_km2 = sup_utiles_ha/100
clustering_data['superficie_agricola_ratio'] = np.where(clustering_data['superficie_km2'] != 0, sup_utiles_agric_km2 / clustering_data['superficie_km2'], 0)

sup_utiles_indust_km2 = clustering_data['sup_total_indust']/1000
clustering_data['superficie_industrial_ratio'] = np.where(clustering_data['superficie_km2'] != 0, sup_utiles_agric_km2 / clustering_data['superficie_km2'], 0)

# Ratio de ocupados y unidades productivas por sector
for sector in ['primario', 'secundario', 'terciario']:
    clustering_data[f'empleo_{sector}_ratio'] = np.where(clustering_data[f'ocupados_{sector}_2023'] != 0, clustering_data[f'ocupados_{sector}_2023'] / clustering_data[f'unidades_productivas_{sector}_2023'], 0)


C:\Users\ana\AppData\Local\Temp\ipykernel_2604\2031130272.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clustering_data['superficie_agricola_ratio'] = np.where(clustering_data['superficie_km2'] != 0, sup_utiles_agric_km2 / clustering_data['superficie_km2'], 0)
C:\Users\ana\AppData\Local\Temp\ipykernel_2604\2031130272.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clustering_data['superficie_industrial_ratio'] = np.where(clustering_data['superficie_km2'] != 0, sup_utiles_agric_km2 / clustering_dat

In [16]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Normalizar los datos
scaler = StandardScaler()
normalized_data = scaler.fit_transform(clustering_data)

# Aplicar K-Means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(normalized_data)
cluster_labels = kmeans.labels_

# Interpretar los resultados
centroides = scaler.inverse_transform(kmeans.cluster_centers_)
centroides_df = pd.DataFrame(centroides, columns=selected_columns)
print(centroides_df)

   contratos_primario_temp_2023  contratos_primario_indefinidos_2023  \
0                      5.490196                             8.607843   
1                     82.000000                            37.857143   
2                     24.611111                            17.500000   

   contratos_primario_total_2023  contratos_secundario_temp_2023  \
0                      14.098039                       83.117647   
1                     119.857143                     3727.285714   
2                      42.111111                      875.777778   

   contratos_secundario_indefinidos_2023  contratos_secundario_total_2023  \
0                             106.928105                       190.045752   
1                            4018.428571                      7745.714286   
2                            1589.388889                      2465.166667   

   contratos_terciario_temp_2023  contratos_terciario_indefinidos_2023  \
0                     433.928105                       

c:\Users\ana\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [17]:
# Etiquetar cada fila de datos con la etiqueta del cluster al que pertenece
clustering_data['cluster'] = cluster_labels

# Agrupar los datos por etiqueta de cluster
cluster_groups = clustering_data.groupby('cluster')

# Calcular estadísticas descriptivas para cada grupo
cluster_stats = cluster_groups[selected_columns].describe()

# Imprimir estadísticas descriptivas para cada característica en cada cluster
print(cluster_stats)

# Interpretar los resultados
for cluster_label, cluster_data in cluster_groups:
    print(f"\nCluster {cluster_label}:")
    # Aquí podríamos interpretar los resultados para determinar qué sector económico predomina en cada cluster
    # Podemos considerar, por ejemplo, la proporción de contratos indefinidos y la cantidad de personas ocupadas en cada sector


C:\Users\ana\AppData\Local\Temp\ipykernel_2604\3568981363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clustering_data['cluster'] = cluster_labels


        contratos_primario_temp_2023                                           \
                               count       mean         std   min   25%   50%   
cluster                                                                         
0                              153.0   5.490196   13.312103   0.0   0.0   0.0   
1                                7.0  82.000000  100.113269  24.0  39.0  48.0   
2                               18.0  24.611111   25.795893   0.0   6.5  17.5   

                     contratos_primario_indefinidos_2023             ...  \
          75%    max                               count       mean  ...   
cluster                                                              ...   
0         4.0   85.0                               153.0   8.607843  ...   
1        58.5  307.0                                 7.0  37.857143  ...   
2        32.5   91.0                                18.0  17.500000  ...   

        unidades_productivas_secundario_2023          \


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Seleccionar las características para el modelo de regresión
features = ['contratos_primario_temp_2023', 'contratos_primario_indefinidos_2023', 
            'contratos_secundario_temp_2023', 'contratos_secundario_indefinidos_2023',
            'contratos_terciario_temp_2023', 'contratos_terciario_indefinidos_2023']

# Seleccionar la variable objetivo
target = 'sector_principal'

# Dividir los datos en conjuntos de entrenamiento y prueba
X = municipios_data[features]
y = municipios_data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo de regresión logística
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluar el modelo
train_accuracy = accuracy_score(y_train, model.predict(X_train))
test_accuracy = accuracy_score(y_test, model.predict(X_test))
print("Precisión en el conjunto de entrenamiento:", train_accuracy)
print("Precisión en el conjunto de prueba:", test_accuracy)

# Predicción del sector principal para cada municipio
predictions = model.predict(X)
municipios_data['prediccion_sector_principal'] = predictions

# Mostrar los primeros municipios con su sector principal predicho
print(municipios_data[['Nombre', 'prediccion_sector_principal']].head())
